In [ ]:
import sys
sys.path.append("..")

from pytorch.data import crop_upper_part, HingeDataset
from pytorch.model import LModel, HingeModel
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn import metrics
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

In [ ]:
ANNOTATED_TEST_DATASET = '../data/test_annotated'
MODEL_PATH = "../models/epoch_19-valLoss_0.01087-valAcc_0.99755"
CPU_CORES = 9
BATCH_SIZE = 64
NUM_CLASSES = 25
USE_GPU = True

In [ ]:
# THIS REQUIRES THE master branch of the `torchivsion` package
def data_transformations(input_shape, crop_perc = 0.5):
    return transforms.Compose([
        transforms.Lambda(lambda x: crop_upper_part(np.array(x), crop_perc)),
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.Resize((input_shape[1], input_shape[2])),
        transforms.ToTensor(),
    ])

def var(tensor, volatile=False):
    if USE_GPU:
        tensor = tensor.cuda(0)
    return Variable(tensor, volatile=volatile)

def test_sigmoid(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.sigmoid(logit).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), np.array(ground_truths)


def test_softmax(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.softmax(logit, dim=0).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), np.array(ground_truths)
    

In [ ]:
# Load model
model_sigmoid = HingeModel(fine_tune=True)


model_sigmoid.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)

if USE_GPU:
        model_sigmoid.cuda(0)

In [ ]:
test_transform = data_transformations(model.model.input_size)
test_dataset = HingeDataset(images_dir=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=CPU_CORES)

## Other vs. All

In [ ]:
pred, true = test_sigmoid(model, test_dataset_loader)

In [ ]:
pred[0:10]

In [ ]:
thr = 0.05
sigmoid_classes = np.copy(pred)
sigmoid_classes[pred > thr] = 1
sigmoid_classes[pred <= thr] = 0

report = metrics.classification_report(sigmoid_classes, true)
print(report)

## Predict 25 class softmax

In [ ]:
thr = 0.9
predicted = []
for act in pred:
    sort = sorted(act)[::-1]
    
    if sort[0] < thr:
        predicted.append(25)
    else:
        predicted.append(np.argmax(act))
        
report = metrics.classification_report(true, predicted)
print(report)